# VDEH Data Loading Pipeline

**Fokus:** Reine Datenextraktion aus OAI-PMH XML-Format

## 🎯 Ziel
- Parse VDEH XML-Datei (OAI-PMH Format)
- Extrahiere Grunddaten: Titel, Autoren, Jahr
- Validiere und bereinige Basisdaten
- Export als standardisierte Parquet-Datei

## 📚 Input/Output
- **Input**: `data/vdeh/raw/VDEH_mab_all.xml`
- **Output**: `data/vdeh/processed/01_loaded_data.parquet`

In [1]:
# 🛠️ SETUP: Konfiguration und Module laden
import sys
import os
from pathlib import Path
import pandas as pd
import warnings

# Projektroot finden
current_dir = Path.cwd()
project_root = None

# Suche nach config.yaml in Parent-Verzeichnissen
for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'config.yaml').exists():
        project_root = parent
        break

if project_root is None:
    raise FileNotFoundError("config.yaml nicht gefunden!")

print(f"📁 Projektroot: {project_root}")

# Python-Pfad erweitern
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"🔧 Python-Pfad erweitert: {src_path}")

📁 Projektroot: /media/sz/Data/Bibo/analysis
🔧 Python-Pfad erweitert: /media/sz/Data/Bibo/analysis/src


In [2]:
# 📋 KONFIGURATION LADEN
import importlib

# Reload config_loader falls schon importiert
if 'config_loader' in sys.modules:
    importlib.reload(sys.modules['config_loader'])
    
from config_loader import load_config

config = load_config(project_root / 'config.yaml')
print("✅ Konfiguration geladen")

# VDEH spezifische Konfiguration
vdeh_config = config.get('data_sources.vdeh')
print(f"📄 VDEH Datenquelle: {vdeh_config['description']}")
print(f"📂 Dateipfad: {config.project_root / vdeh_config['path']}")
print(f"🔤 Encoding: {vdeh_config['encoding']}")
print(f"📊 Geschätzte Records: {vdeh_config['estimated_records']:,}")

# Warnings konfigurieren
if not config.get('debug.verbose_output', True):
    warnings.filterwarnings('ignore')

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
✅ Konfiguration geladen
📄 VDEH Datenquelle: Neuerwerbungen der VDEH Bibliotheken
📂 Dateipfad: /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml
🔤 Encoding: utf-8
📊 Geschätzte Records: 58,760


In [3]:
# 📊 XML-DATEN LADEN
vdeh_config = config.get('data_sources.vdeh')
xml_file_path = config.project_root / vdeh_config['path']
max_records = config.get('data_processing.vdeh_parser.max_records')

print(f"📂 Lade VDEH XML-Datei: {xml_file_path}")
print(f"📊 Max Records: {max_records or 'Alle (~58k)'}")

# Prüfe ob Datei existiert
if not xml_file_path.exists():
    raise FileNotFoundError(f"VDEH XML-Datei nicht gefunden: {xml_file_path}")

# VDEH Parser importieren
from parsers.vdeh_parser import parse_bibliography, analyze_bibliography_data, get_sample_records

# Daten laden
df_vdeh = parse_bibliography(str(xml_file_path), max_records=max_records)

print(f"\n✅ DataFrame erstellt: {len(df_vdeh):,} Records")
print(f"💾 Speicherverbrauch: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"📋 Spalten: {list(df_vdeh.columns)}")

📂 Lade VDEH XML-Datei: /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml
📊 Max Records: Alle (~58k)
🚀 Starte robusten Parser für bibliographische Grunddaten...
📁 Datei: /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml
📊 Dateigröße: 118.6 MB
📚 Gefunden: 58,760 OAI-Records
📝 Verarbeitet: 5,000 Records
📝 Verarbeitet: 10,000 Records
📝 Verarbeitet: 15,000 Records
📝 Verarbeitet: 20,000 Records
📝 Verarbeitet: 25,000 Records
📝 Verarbeitet: 30,000 Records
📝 Verarbeitet: 35,000 Records
📝 Verarbeitet: 40,000 Records
📝 Verarbeitet: 45,000 Records
📝 Verarbeitet: 50,000 Records
📝 Verarbeitet: 55,000 Records
✅ 58,760 Records verarbeitet
🎯 DataFrame erstellt:
   📊 58,760 Zeilen
   📋 12 Spalten
   💾 31.9 MB

✅ DataFrame erstellt: 58,760 Records
💾 Speicherverbrauch: 31.9 MB
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation']


In [8]:
df_vdeh.iloc[0]           

id                                                   aleph-publish:000000004
title                      Untersuchung der Gleichgewichte zwischen flüss...
authors                                                      [Thielmann, R.]
authors_affiliation                                 [Europäische Kommission]
year                                                                  1983.0
publisher                                                         Luxembourg
isbn                                                                    None
issn                                                                    None
authors_str                                                    Thielmann, R.
num_authors                                                                1
authors_affiliation_str                               Europäische Kommission
num_authors_affiliation                                                    1
Name: 0, dtype: object

In [5]:
# Standardnummern-Analyse (ISBN/ISSN)
print("=== STANDARDNUMMERN ANALYSE ===")

# ISBN Analyse
try:
    isbn_total = df_vdeh['isbn'].notna().sum()
    print("\nISBN Analyse:")
    print(f"Anzahl Datensätze mit ISBN: {isbn_total:,}")
    print(f"Prozent der Datensätze mit ISBN: {(isbn_total/len(df_vdeh)*100):.2f}%")

    if isbn_total > 0:
        # ISBN Längenanalyse
        df_with_isbn = df_vdeh[df_vdeh['isbn'].notna()]
        if not df_with_isbn.empty:
            isbn_lengths = df_with_isbn['isbn'].astype(str).str.len().value_counts().sort_index()
            print("\nISBN Längen:")
            for length, count in isbn_lengths.items():
                print(f"   {length} Stellen: {count:,} ISBNs ({count/isbn_total*100:.1f}%)")

            print("\nBeispiel ISBNs:")
            try:
                sample_with_isbn = df_with_isbn.sample(n=min(3, len(df_with_isbn)), random_state=42)
                for _, row in sample_with_isbn.iterrows():
                    print(f"   ISBN: {row['isbn']} | Titel: {row['title'][:70] if row['title'] else 'N/A'}...")
            except ValueError as e:
                print("   Keine ISBN-Beispiele verfügbar.")
except Exception as e:
    print(f"Fehler bei der ISBN-Analyse: {str(e)}")

# ISSN Analyse
try:
    issn_total = df_vdeh['issn'].notna().sum()
    print("\nISSN Analyse:")
    print(f"Anzahl Datensätze mit ISSN: {issn_total:,}")
    print(f"Prozent der Datensätze mit ISSN: {(issn_total/len(df_vdeh)*100):.2f}%")

    if issn_total > 0:
        df_with_issn = df_vdeh[df_vdeh['issn'].notna()]
        if not df_with_issn.empty:
            print("\nBeispiel ISSNs:")
            try:
                sample_with_issn = df_with_issn.sample(n=min(3, len(df_with_issn)), random_state=42)
                for _, row in sample_with_issn.iterrows():
                    print(f"   ISSN: {row['issn']} | Titel: {row['title'][:70] if row['title'] else 'N/A'}...")
            except ValueError as e:
                print("   Keine ISSN-Beispiele verfügbar.")
except Exception as e:
    print(f"Fehler bei der ISSN-Analyse: {str(e)}")

# Verlagsanalyse
if 'publisher' in df_vdeh.columns:
    try:
        publisher_total = df_vdeh['publisher'].notna().sum()
        print("\n\n=== VERLAGSANALYSE ===")
        print(f"Anzahl Datensätze mit Verlag: {publisher_total:,}")
        print(f"Prozent der Datensätze mit Verlag: {(publisher_total/len(df_vdeh)*100):.2f}%")
        
        if publisher_total > 0:
            print("\nBeispiel Verlage:")
            df_with_publisher = df_vdeh[df_vdeh['publisher'].notna()]
            sample_publisher = df_with_publisher.sample(n=min(3, len(df_with_publisher)), random_state=42)
            for _, row in sample_publisher.iterrows():
                print(f"   Verlag: {row['publisher'][:60]}... | Titel: {row['title'][:50] if row['title'] else 'N/A'}...")
    except Exception as e:
        print(f"Fehler bei der Verlagsanalyse: {str(e)}")

=== STANDARDNUMMERN ANALYSE ===

ISBN Analyse:
Anzahl Datensätze mit ISBN: 11,415
Prozent der Datensätze mit ISBN: 19.43%

ISBN Längen:
   0 Stellen: 1 ISBNs (0.0%)
   1 Stellen: 1 ISBNs (0.0%)
   4 Stellen: 2 ISBNs (0.0%)
   7 Stellen: 2 ISBNs (0.0%)
   8 Stellen: 697 ISBNs (6.1%)
   9 Stellen: 22 ISBNs (0.2%)
   11 Stellen: 22 ISBNs (0.2%)
   12 Stellen: 13 ISBNs (0.1%)
   13 Stellen: 8,038 ISBNs (70.4%)
   14 Stellen: 1 ISBNs (0.0%)
   16 Stellen: 2 ISBNs (0.0%)
   17 Stellen: 2,485 ISBNs (21.8%)
   20 Stellen: 77 ISBNs (0.7%)
   21 Stellen: 1 ISBNs (0.0%)
   22 Stellen: 5 ISBNs (0.0%)
   23 Stellen: 31 ISBNs (0.3%)
   25 Stellen: 1 ISBNs (0.0%)
   26 Stellen: 9 ISBNs (0.1%)
   27 Stellen: 1 ISBNs (0.0%)
   28 Stellen: 1 ISBNs (0.0%)
   30 Stellen: 3 ISBNs (0.0%)

Beispiel ISBNs:
   ISBN: 3-826-56022-1 | Titel: Tribologische Untersuchungen zur Kaltmassivumformung...
   ISBN: 9-282-81437-8 | Titel: Mejora de las propiedades mecanicas de piezas forjadas y barras median...
   ISBN: 3-5

In [6]:
# 📈 BASIS-DATENANALYSE
print("📈 === VDEH DATENANALYSE ===\n")

# Basis-Analyse der VDEH-Daten
analyze_bibliography_data(df_vdeh)

📈 === VDEH DATENANALYSE ===


📊 === BIBLIOGRAPHISCHE DATEN ANALYSE ===
📚 Gesamt Records: 58,760
📋 Mit Titel: 40,830 (69.5%)
👤 Mit Autoren: 17,011 (28.9%)
📅 Mit Jahr: 33,687 (57.3%)
🏢 Mit Verlag: 23,553 (40.1%)
📖 Mit ISBN: 11,415 (19.4%)
📰 Mit ISSN: 721 (1.2%)
✅ Vollständig: 16,013 (27.3%)
📅 Zeitspanne: 1900 - 2017
📊 Median-Jahr: 1998
👥 Einzigartige Autoren: 17,758


In [7]:
# 💾 GELADENE DATEN EXPORTIEREN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
processed_dir.mkdir(parents=True, exist_ok=True)

# Export als Parquet für nächste Pipeline-Stufe
output_path = processed_dir / '01_loaded_data.parquet'
df_vdeh.to_parquet(output_path, index=False)

print(f"💾 === DATEN LOADING ABGESCHLOSSEN ===")
print(f"✅ Geladene Daten exportiert: {output_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {len(df_vdeh.columns)}")
print(f"💾 Dateigröße: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Metadaten für nächste Stufe
metadata = {
    'stage': '01_data_loading',
    'records_loaded': len(df_vdeh),
    'columns': list(df_vdeh.columns),
    'source_file': str(xml_file_path),
    'processing_date': pd.Timestamp.now().isoformat()
}

import json
metadata_path = processed_dir / '01_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"📄 Metadaten gespeichert: {metadata_path}")
print(f"\n➡️  Nächster Schritt: 02_vdeh_data_preprocessing.ipynb")

💾 === DATEN LOADING ABGESCHLOSSEN ===
✅ Geladene Daten exportiert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/01_loaded_data.parquet
📊 Records: 58,760
📋 Spalten: 12
💾 Dateigröße: 34.8 MB
📄 Metadaten gespeichert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/01_metadata.json

➡️  Nächster Schritt: 02_vdeh_data_preprocessing.ipynb
